- https://github.com/goodarzilab/Ribolog/blob/master/vignettes/RIBOLOG.pdf


In [20]:
# !mkdir bam
# !mkdir bam/RNA
# !mkdir bam/RPF

# !mkdir bam_processed/
# !mkdir bam_processed/RNA
# !mkdir bam_processed/RPF

In [19]:
# cat alignment.sh

    ---
    title: "Decitabine"
    author: "Hossein Asgharian"
    date: "5/13/2020"
    output: pdf_document
    ---

<!-- Hi, Hossin is asking me a question about Riboseq data for Alex's project 
which bam files in the RPF folder do you want me to use?
-->

<!-- *.dd.bam for RPF
and normal for RNA
 -->

<!-- Both RNA and RPF file should always be mapped to the fasta where headers are just transcript IDs. RPF fastqs were mapped to the right fasta but RNAs were mapped to the full header fasta (transcript ID, gene ID, gene name etc). It causes problem during input and merger. I fixed the RNA bams by samtools reheader. -->

https://github.com/goodarzilab/ribosome_footprinting

I need to get Help from Hani and Sohit

In [25]:
library(tidyverse)
library(Ribolog)
library(knitr)
library(riboWaltz)
library(ggplot2)

# opts_chunk$set(tidy.opts=list(width.cutoff=60),tidy=TRUE)
# opts_chunk$set(root.dir = "C:/Science/Projects/Ribosome profiling/People/Abe/")
# library(xlsx)
# library(EnhancedVolcano)

source('../util.R')

In [62]:
annotation_human_cDNA <- Ribolog::read_annotation("~/genomes/Ribolog/Human.GRC38.96_annotation.txt")

# Input RPF data

In [63]:
reads_list_deci <- Ribolog::bamtolist_rW(bamfolder = "./bam/RPF", annotation = annotation_human_cDNA)

Reading 1_RiboSeq_S25DMSO_r1.ensemble.cds.longest.srt.dd.bam
Input reads: 2.187 M
0.000 M  (0.013 %) reads removed: exceeding indel_threshold.
0.325 M  (14.857 %) reads removed: reference transcript IDs not found in annotation table.
0.269 M  (14.450 %) reads removed: mapping on negative strand.
Output reads: 1.593 M
Done! 1_RiboSeq_S25DMSO_r1.ensemble.cds.longest.srt.dd.bam has been loaded as 1_RiboSeq_S25DMSO_r1.ensemble.cds.longest.srt.dd 

Reading 1_RiboSeq_S25DMSO_r2.ensemble.cds.longest.srt.dd.bam
Input reads: 1.451 M
0.000 M  (0.014 %) reads removed: exceeding indel_threshold.
0.219 M  (15.092 %) reads removed: reference transcript IDs not found in annotation table.
0.165 M  (13.387 %) reads removed: mapping on negative strand.
Output reads: 1.067 M
Done! 1_RiboSeq_S25DMSO_r2.ensemble.cds.longest.srt.dd.bam has been loaded as 1_RiboSeq_S25DMSO_r2.ensemble.cds.longest.srt.dd 

Reading 1_RiboSeq_S25Drug_r1.ensemble.cds.longest.srt.dd.bam
Input reads: 1.484 M
0.000 M  (0.013 %) rea

#### rename

In [64]:
names(reads_list_deci)

[1] "1_RiboSeq_S25DMSO_r1.ensemble.cds.longest.srt.dd"
[2] "1_RiboSeq_S25DMSO_r2.ensemble.cds.longest.srt.dd"
[3] "1_RiboSeq_S25Drug_r1.ensemble.cds.longest.srt.dd"
[4] "1_RiboSeq_S25Drug_r2.ensemble.cds.longest.srt.dd"

In [65]:
names(reads_list_deci) <- c(
    "DMSO_rep1_RPF", "DMSO_rep2_RPF", 
    "Decitabine_rep1_RPF", "Decitabine_rep2_RPF"
)

In [66]:
psite_offset_deci <- Ribolog::psite_rW(reads_list_deci)

processing DMSO_rep1_RPF
best offset: 12 nts from the 5' end
processing DMSO_rep2_RPF
best offset: 13 nts from the 5' end
processing Decitabine_rep1_RPF
best offset: 13 nts from the 5' end
processing Decitabine_rep2_RPF
best offset: 13 nts from the 5' end


In [67]:
reads_psite_list_deci <- Ribolog::psite_info_rW(reads_list_deci, psite_offset_deci)

processing DMSO_rep1_RPF
1. adding p-site position
2. adding transcript region
processing DMSO_rep2_RPF
1. adding p-site position
2. adding transcript region
processing Decitabine_rep1_RPF
1. adding p-site position
2. adding transcript region
processing Decitabine_rep2_RPF
1. adding p-site position
2. adding transcript region


### Check periodicity and choose readlength range

In [68]:
Ribolog::print_read_ldist(reads_list_deci, "./plots/deci_RPF_Read_length_distributions.pdf")

Warning message:
“Removed 8 rows containing missing values (position_stack).”
Warning message:
“Removed 8 rows containing missing values (position_stack).”
Warning message:
“Removed 7 rows containing missing values (position_stack).”
Warning message:
“Removed 7 rows containing missing values (position_stack).”


[1] "PDF (./plots/deci_RPF_Read_length_distributions.pdf) created and saved"

In [69]:
Ribolog::print_period_region(reads_psite_list_deci, "./plots/Periodicity_by_region.pdf")

[1] "PDF (./plots/Periodicity_by_region.pdf) created and saved"

In [72]:
Ribolog::print_period_region_length(reads_psite_list_deci, "./plots/Periodicity_by_length_region.pdf")

[1] "PDF (./plots/Periodicity_by_length_region.pdf) created and saved"

In [20]:
# Ribolog::print_read_end_heatmap(reads_list_deci, annotation_human_cDNA, "./plots/deci_RPF_read_end_heatmaps.pdf")

In [100]:
# Ribolog::print_rop(reads_psite_list_deci, annotation_human_cDNA, "./plots/RPF_ribosome_occupancy_profiles_from_annotation.pdf")

Check what read lengths will be retained by a periodicity filter:



In [73]:
filtered_list_deci <- length_filter(data = reads_list_deci, length_filter_mode = "periodicity", periodicity_threshold = 40)

processing DMSO_rep1_RPF
reads: 1.59 M
0.89 M  (55.84 %) reads removed
reads (kept): 0.70 M

processing DMSO_rep2_RPF
reads: 1.07 M
0.60 M  (56.24 %) reads removed
reads (kept): 0.47 M

processing Decitabine_rep1_RPF
reads: 1.07 M
0.62 M  (58.33 %) reads removed
reads (kept): 0.44 M

processing Decitabine_rep2_RPF
reads: 1.01 M
0.64 M  (63.42 %) reads removed
reads (kept): 0.37 M



With the default of threshold = 50, all reads were removed. Threshold = 40 still removes 60-70% of reads.


In [30]:
# lapply(filtered_list_deci, function(x) summary(as.factor(x$length))) 

From this output and the plots generated above, the choice of read length 29-40 seems reasonable. 


In [39]:
# tr_codon_read_count_deci <- Ribolog::psite_to_codon_count(reads_psite_list_deci, c(29:40), annotation_human_cDNA,  "~/genomes/Ribolog/Human.GRC38.96_cDNA_longest_CDS.txt")

In [41]:
# tr_codon_bias_coeff_loess_corrected_count_deci <- Ribolog::CELP_bias(tr_codon_read_count_deci, loess_method = "direct")


In [43]:
# rpf_corrected_sum_deci <- Ribolog::codon2transcript(tr_codon_bias_coeff_loess_corrected_count_deci$tr_codon_read_count_loess_corrected, count.type = "corrected_count")
# head(rpf_corrected_sum_deci)


In [45]:
# rpf_observed_sum_deci <- Ribolog::codon2transcript(tr_codon_bias_coeff_loess_corrected_count_deci$tr_codon_read_count_loess_corrected, count.type = "observed_count")
# head(rpf_observed_sum_deci)

# Input RNA data

In [47]:
rna_count_deci <- Ribolog::bam2count(bamfolder = "./bam/RNA", annotation = annotation_human_cDNA)

In [ ]:
rna_CELP_rpf_count_deci <- merge(rna_count_deci, rpf_corrected_sum_deci, by = "transcript")
head(rna_CELP_rpf_count_deci)

Rename columns for convenience.

In [ ]:
names(rna_CELP_rpf_count_deci)[c(2:5)] <- c("dmso_r1_rna", "dmso_r2_rna", "drug_r1_rna", "drug_r2_rna")
head(rna_CELP_rpf_count_deci)

This is now ready for normalization and filtering.


In [ ]:
# Normalize RNA counts
rna_CELP_rpf_count_norm1_deci <- Ribolog::normalize_median_of_ratios(rna_CELP_rpf_count_deci, c(2:5))
# Normalize RPF counts
rna_CELP_rpf_count_norm2_deci <- Ribolog::normalize_median_of_ratios(rna_CELP_rpf_count_norm1_deci, c(6:9))



In [ ]:
# Filter for RNA>=5 in all samples
rr_deci.v1 <- Ribolog::min_count_filter(rna_CELP_rpf_count_norm2_deci, mincount = 10, columns = c(2:5), method = "all") 
# Filter for average RPF>=5 across samples
rr_deci.v2 <- Ribolog::min_count_filter(rr_deci.v1, mincount = 5, columns = c(6:9), method = "average") 
dim(rna_CELP_rpf_count_norm2_deci)
dim(rr_deci.v1)
dim(rr_deci.v2)


### QC


In [ ]:
te_deci.v2 <- Ribolog::create_te(rr_deci.v2, idcolumns = 1, rnacolumns = c(2:5), rpfcolumns = c(6:9))
head(te_deci.v2)



In [ ]:
te_deci.v2.cent <- row_center(te_deci.v2, columns = c(2:5))
te_deci.v2.stnd <- row_standardize(te_deci.v2, columns = c(2:5))
sample_attributes_deci <- read.xlsx("./sample_attributes_deci.xlsx", sheetIndex = 1, header = TRUE)
print(sample_attributes_deci)
Ribolog::pca_qc(te_deci.v2[, -1], n = 2, ID = sample_attributes_deci$treatment[c(1:4)])
Ribolog::pca_qc(te_deci.v2.cent[, -1], n = 2, ID = sample_attributes_deci$treatment[c(1:4)])
Ribolog::pca_qc(te_deci.v2.stnd[, -1], n = 2, ID = sample_attributes_deci$treatment[c(1:4)])

In [ ]:
rr_deci.v2.split <- Ribolog::partition_to_uniques(x = rr_deci.v2[,-1], design = sample_attributes_deci, uniqueID = "replicate_name")
names(rr_deci.v2.split)

In [ ]:
rr_deci.v2.pairwise <- Ribolog::TER_all_pairs(x = rr_deci.v2.split, design = sample_attributes_deci, outcome = "read_type", uniqueID = "replicate_name", groupID = "treatment")
pi0df_deci <- Ribolog::pairs2pi0s(rr_deci.v2.pairwise)
print(pi0df_deci)


In [ ]:
rr_deci.v2.correlograms <- pairs2correlograms(rr_deci.v2.pairwise)



In [ ]:
fit1_deci <- Ribolog::logit_seq(rr_deci.v2[,-1], sample_attributes_deci, read_type ~ treatment, as.vector(rr_deci.v2$transcript)) 
head(fit1_deci)
hist(fit1_deci[,4], n=50, main = "p-value distribution", xlab = "p-value")



P-value distribution looks good.


In [ ]:
pdf("./QC and volcano plots.pdf")
Ribolog::pca_qc(te_deci.v2[, -1], n = 2, ID = sample_attributes_deci$treatment[c(1:4)])
Ribolog::pca_qc(te_deci.v2.cent[, -1], n = 2, ID = sample_attributes_deci$treatment[c(1:4)])
Ribolog::pca_qc(te_deci.v2.stnd[, -1], n = 2, ID = sample_attributes_deci$treatment[c(1:4)])
pi0df_deci <- Ribolog::pairs2pi0s(rr_deci.v2.pairwise)
rr_deci.v2.correlograms <- pairs2correlograms(rr_deci.v2.pairwise)
print(vplot2)
dev.off()

In [ ]:
fit1_deci_FDR <- Ribolog::adj_TER_p(fit1_deci, pcols = 4, adj_method = "fdr")
fit1_deci_qval <- Ribolog::adj_TER_p(fit1_deci, pcols = 4, adj_method = "qvalue")
fit1_deci_fdr_qval <- Ribolog::adj_TER_p(fit1_deci_FDR, pcols = 4, adj_method = "qvalue")
fit1_deci_fdr_qval$transcript <- rownames(fit1_deci_fdr_qval)
fit1_deci_fdr_qval_g <- merge(human_id_mapper, fit1_deci_fdr_qval, by = "transcript")
write.table(fit1_deci_fdr_qval_g, "C:/Science/Projects/Ribosome profiling/People/Abe/fit1_deci_fdr_qval_g.txt", row.names = FALSE)
write.csv(fit1_deci_fdr_qval_g, "C:/Science/Projects/Ribosome profiling/People/Abe/fit1_deci_fdr_qval_g.csv", row.names = FALSE)

In [ ]:
# ```{r message = FALSE, warning = FALSE, fig.width = 6, fig.height = 7}
vplot1 <- EnhancedVolcano(fit1_deci_fdr_qval_g,
                lab = fit1_deci_fdr_qval_g$gene_name,
                x = 'Estimate_treatmentDRUG',
                xlab = 'Ln fold change',
                y = 'fdr_Pr...z.._treatmentDRUG',
                ylab = '-Log10 FDR',
                title = 'Decitabine data, Drug vs DMSO',
                titleLabSize = 12,
                border = 'full',
                pCutoff = 0.05,
                FCcutoff = log(2),
                xlim = c(-5, 5), ylim = c(0, 10)) 
vplot1

# ```{r message = FALSE, warning = FALSE, fig.width = 6, fig.height = 7}
pdf("./Decitabine treatment effect volcano plot, qvalue.pdf")
vplot2 <- EnhancedVolcano(fit1_deci_fdr_qval_g,
                lab = fit1_deci_fdr_qval_g$gene_name,
                x = 'Estimate_treatmentDRUG',
                xlab = 'Ln fold change',
                y = 'qvalue_Pr...z.._treatmentDRUG',
                ylab = '-Log10 q value',
                title = 'Decitabine data, Drug vs DMSO',
                titleLabSize = 12,
                border = 'full',
                pCutoff = 0.05,
                FCcutoff = log(2),
                xlim = c(-3, 3), ylim = c(0, 6)) 
vplot2
dev.off()